In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import altair as alt
import time
import sklearn
import warnings
import pickle

warnings.filterwarnings(action='ignore', category=FutureWarning)

pd.__version__, sklearn.__version__

('2.1.1', '1.3.1')

In [2]:
SCRATCH_DIR = "/scratch/siads696f23_class_root/siads696f23_class/psollars"

# For local dev
SCRATCH_DIR = "./../data"

In [3]:
# Checkpoint

# consolidated_df.to_parquet(f"{SCRATCH_DIR}/consolidated_features_all_delays.parquet")

consolidated_df = pd.read_parquet(f"{SCRATCH_DIR}/consolidated_features_all_delays.parquet")

In [4]:
consolidated_df_num_cols = [
    "Quarter",
    "Month",
    "DayofMonth",
    "DayOfWeek",
    "CRSDepTime",
    "CRSArrTime",
    "CRSElapsedTime",
    "Distance",
    "DistanceGroup",
    "YEAR MFR",
    "HORSEPOWER",
    "THRUST",
    "NO-SEATS",
    "Origin_LATITUDE",
    "Origin_LONGITUDE",
    "Dest_LATITUDE",
    "Dest_LONGITUDE",
]

consolidated_df_cat_cols = [
    "Reporting_Airline",
    "Tail_Number",
    "Origin",
    "Dest",
    "TYPE REGISTRANT",
    "NAME",
    "CERTIFICATION",
    "TYPE ENGINE",
    "STATUS CODE",
    "MFR",
    "MODEL",
    "TYPE",
    "MFR_aircraft",
    "MODEL_aircraft",
    "BUILD-CERT-IND",
    #
    # Labels only, duplicates of the airport coordinates, also Origin/Dest
    # "Origin_DISPLAY_AIRPORT_NAME",
    # "Origin_DISPLAY_AIRPORT_CITY_NAME_FULL",
    # "Dest_DISPLAY_AIRPORT_NAME",
    # "Dest_DISPLAY_AIRPORT_CITY_NAME_FULL",
    #
    # Only concerned with CarrierDelay for this round
    # "DepDel15",
    # "ArrDel15",
    # "CarrierDelay",
    # "WeatherDelay",
    # "NASDelay",
    # "SecurityDelay",
    # "LateAircraftDelay",
]

In [6]:
united_df = consolidated_df[consolidated_df_num_cols + consolidated_df_cat_cols + ["DepDel15", "ArrDel15"]].copy()

united_df = united_df[united_df["Reporting_Airline"].eq("UA")]
united_df = united_df[united_df["Origin"].eq("ORD") | united_df["Dest"].eq("ORD")]

# Convert this to "delayed"
united_df["delayed"] = (united_df["DepDel15"].eq(1)) | (united_df["ArrDel15"].eq(1))
united_df = united_df.drop(["DepDel15", "ArrDel15"], axis=1)

# Drop NaNs
united_df = united_df.dropna(how="any", axis="index")

# Drop any features that have all identical values
nunique = united_df.nunique()
united_df = united_df.drop(nunique[nunique.eq(1)].index, axis=1)
united_df["TYPE ENGINE"] = united_df["TYPE ENGINE"].astype(int)
united_df["TYPE"] = united_df["TYPE"].astype(int)

# These columns are confusingly named
united_df = united_df.rename(
    columns={
        "TYPE ENGINE": "engine_type",
        "MFR": "engine_manufacturer",
        "MODEL": "engine_model",
        "TYPE": "aircraft_type"
        "MFR_aircraft": "aircraft_manufacturer",
        "MODEL_aircraft": "aircraft_model",
    }
)

print(united_df.columns)
print(united_df.value_counts("delayed"))
# delayed
# False    114157
# True      39403

united_df
# 153560 rows × 29 columns

Index(['Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime',
       'CRSArrTime', 'CRSElapsedTime', 'Distance', 'DistanceGroup', 'YEAR MFR',
       'HORSEPOWER', 'THRUST', 'NO-SEATS', 'Origin_LATITUDE',
       'Origin_LONGITUDE', 'Dest_LATITUDE', 'Dest_LONGITUDE', 'Tail_Number',
       'Origin', 'Dest', 'NAME', 'CERTIFICATION', 'engine_type',
       'engine_manufacturer', 'engine_model', 'aircraft_type',
       'aircraft_manufacturer', 'aircraft_model', 'delayed'],
      dtype='object')
delayed
False    114157
True      39403
Name: count, dtype: int64


,Quarter,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,CRSElapsedTime,Distance,DistanceGroup,YEAR MFR,...,Dest,NAME,CERTIFICATION,engine_type,engine_manufacturer,engine_model,aircraft_type,aircraft_manufacturer,aircraft_model,delayed
25272188,1,1,22,2,1315,1932,257,1846,8,2001.0,...,ORD,WELLS FARGO TRUST CO NA TRUSTEE ...,1T,5,ROLLS-ROYCE,RB-211 SERIES,5,BOEING,757-33N,True
25272196,1,1,22,2,1000,1144,164,733,3,2002.0,...,ORD,UNITED AIRLINES INC ...,1T,5,CFM INTL,CFM56-7B22,5,BOEING,737-700,False
25272197,1,1,22,2,1430,1650,80,296,2,2015.0,...,CMH,UNITED AIRLINES INC ...,1T,5,CFM INTL,CFM56-7B27E,5,BOEING,737-924ER,True
25272202,1,1,22,2,1011,1200,169,888,4,2004.0,...,DEN,UNITED AIRLINES INC ...,1T,5,ROLLS-ROYCE,RB-211 SERIES,5,BOEING,757-324,True
25272222,1,1,22,2,1423,2024,241,1723,7,2004.0,...,ORD,UMB BANK NA TRUSTEE ...,1T,5,CFM INTL,CFM56 SERIES,5,BOEING,737-824,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32219984,4,12,28,6,1747,2014,207,1197,5,1994.0,...,ORD,U S BANK NA TRUSTEE ...,1T,5,IAE,V2500SERIES,4,AIRBUS,A320-232,True
32219989,4,12,28,6,0755,1049,174,925,4,2014.0,...,IAH,UNITED AIRLINES INC ...,1T,5,CFM INTL,CFM56-7B27E,5,BOEING,737-924ER,False
32219991,4,12,28,6,2324,0510,226,1739,7,1999.0,...,ORD,UNITED AIRLINES INC ...,1T,5,CFM INTL,CFM56 SERIES,5,BOEING,737-824,False
32219994,4,12,28,6,0745,1145,180,1120,5,2008.0,...,RSW,UNITED AIRLINES INC ...,1T,5,CFM INTL,CFM56-7B26,5,BOEING,737-924ER,False


In [7]:
# Checkpoint

united_df.to_parquet(f"{SCRATCH_DIR}/model_selection_all_delays_ORD_UA.parquet")

# united_df = pd.read_parquet(f"{SCRATCH_DIR}/model_selection_all_delays_ORD_UA.parquet")

In [6]:
# Define features one last time now that the identical values have been removed

num_cols = [
    "Quarter",
    "Month",
    "DayofMonth",
    "DayOfWeek",
    "CRSDepTime",
    "CRSArrTime",
    "CRSElapsedTime",
    "Distance",
    "DistanceGroup",
    "YEAR MFR",
    "HORSEPOWER",
    "THRUST",
    "NO-SEATS",
    "Origin_LATITUDE",
    "Origin_LONGITUDE",
    "Dest_LATITUDE",
    "Dest_LONGITUDE",
]

cat_cols = [
    "Tail_Number",
    "Origin",
    "Dest",
    "NAME",
    "CERTIFICATION",
    "engine_type",
    "engine_manufacturer",
    "engine_model",
    "aircraft_type" "aircraft_manufacturer",
    "aircraft_model",
]

In [7]:
# !pip install catboost

import catboost

catboost.__version__

'1.2.2'

In [8]:
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split


# No need to use the ordinal encoder for CatBoost, but we need to do some other preprocessing
X = united_df.drop("delayed", axis=1)
y = united_df["delayed"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

cat_indices = [X.columns.get_loc(col) for col in cat_cols]

print(cat_indices)

[17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]


In [9]:
start_time = time.time()

smote = SMOTENC(categorical_features=cat_indices, sampling_strategy="auto", random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

end_time = time.time()

print(f"Elapsed time: {(end_time - start_time):.4f} seconds")

X_res.shape, y_res.shape

Elapsed time: 553.5295 seconds


((182596, 28), (182596,))

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, fbeta_score


def fit_grid_search_cat_boost(fit_X_train, fit_y_train):
    cat_clf = CatBoostClassifier(
        eval_metric="F1",
        auto_class_weights="Balanced",
        cat_features=cat_indices,
        verbose=0,
        task_type="GPU"
    )

    # Attempt to increase precision
    f_beta_scorer = make_scorer(fbeta_score, beta=0.5)

    grid_search = GridSearchCV(
        estimator=cat_clf,
        param_grid={
            "learning_rate": [0.001, 0.01, 0.1, 0.2],
            "depth": [6, 8, 10],
            "iterations": [100, 200],
            "l2_leaf_reg": [1, 3, 5],
        },
        scoring=f_beta_scorer,
        verbose=3,
        cv=5,
        n_jobs=-1,
    )

    grid_search.fit(fit_X_train, fit_y_train)

    print("Best parameters: ", grid_search.best_params_)
    print("Best score: ", grid_search.best_score_)

    return grid_search


start_time = time.time()

model = fit_grid_search_cat_boost(X_res, y_res)

end_time = time.time()

print(f"Elapsed time: {(end_time - start_time):.4f} seconds")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


TBB Warning: The number of workers is currently limited to 0. The request for 63 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



0:	learn: 0.6552889	total: 33.1ms	remaining: 3.27s
1:	learn: 0.6670205	total: 73ms	remaining: 3.57s
2:	learn: 0.6661712	total: 106ms	remaining: 3.42s
3:	learn: 0.6661181	total: 138ms	remaining: 3.32s
4:	learn: 0.6658525	total: 171ms	remaining: 3.25s
5:	learn: 0.6665792	total: 204ms	remaining: 3.19s
6:	learn: 0.6664432	total: 238ms	remaining: 3.16s
7:	learn: 0.6650268	total: 271ms	remaining: 3.11s
8:	learn: 0.6649739	total: 303ms	remaining: 3.07s
9:	learn: 0.6652394	total: 336ms	remaining: 3.03s
10:	learn: 0.6651222	total: 369ms	remaining: 2.98s
11:	learn: 0.6671667	total: 402ms	remaining: 2.94s
12:	learn: 0.6667322	total: 434ms	remaining: 2.91s
13:	learn: 0.6665794	total: 468ms	remaining: 2.88s
14:	learn: 0.6666449	total: 501ms	remaining: 2.84s
15:	learn: 0.6666449	total: 534ms	remaining: 2.8s
16:	learn: 0.6663611	total: 567ms	remaining: 2.77s
17:	learn: 0.6663611	total: 599ms	remaining: 2.73s
18:	learn: 0.6663351	total: 632ms	remaining: 2.69s
19:	learn: 0.6659271	total: 665ms	remainin

63:	learn: 0.6464807	total: 2.1s	remaining: 1.18s
64:	learn: 0.6465067	total: 2.14s	remaining: 1.15s
65:	learn: 0.6465067	total: 2.17s	remaining: 1.12s
66:	learn: 0.6465067	total: 2.2s	remaining: 1.08s
67:	learn: 0.6464975	total: 2.24s	remaining: 1.05s
68:	learn: 0.6464252	total: 2.27s	remaining: 1.02s
69:	learn: 0.6464304	total: 2.3s	remaining: 987ms
70:	learn: 0.6464401	total: 2.34s	remaining: 954ms
71:	learn: 0.6464503	total: 2.37s	remaining: 921ms
72:	learn: 0.6463967	total: 2.4s	remaining: 888ms
73:	learn: 0.6464252	total: 2.43s	remaining: 855ms
74:	learn: 0.6464352	total: 2.47s	remaining: 822ms
75:	learn: 0.6464401	total: 2.5s	remaining: 789ms
76:	learn: 0.6464543	total: 2.53s	remaining: 756ms
77:	learn: 0.6463915	total: 2.57s	remaining: 724ms
78:	learn: 0.6464358	total: 2.6s	remaining: 691ms
79:	learn: 0.6461602	total: 2.63s	remaining: 658ms
80:	learn: 0.6469951	total: 2.66s	remaining: 625ms
81:	learn: 0.6469951	total: 2.7s	remaining: 592ms
82:	learn: 0.6469558	total: 2.73s	rema

21:	learn: 0.6357606	total: 722ms	remaining: 2.56s
22:	learn: 0.6357606	total: 754ms	remaining: 2.52s
23:	learn: 0.6358141	total: 787ms	remaining: 2.49s
24:	learn: 0.6358141	total: 819ms	remaining: 2.46s
25:	learn: 0.6358141	total: 852ms	remaining: 2.42s
26:	learn: 0.6357606	total: 885ms	remaining: 2.39s
27:	learn: 0.6358141	total: 917ms	remaining: 2.36s
28:	learn: 0.6357606	total: 951ms	remaining: 2.33s
29:	learn: 0.6357606	total: 984ms	remaining: 2.29s
30:	learn: 0.6357606	total: 1.02s	remaining: 2.26s
31:	learn: 0.6357716	total: 1.05s	remaining: 2.23s
32:	learn: 0.6357606	total: 1.08s	remaining: 2.19s
33:	learn: 0.6357606	total: 1.11s	remaining: 2.16s
34:	learn: 0.6357606	total: 1.15s	remaining: 2.13s
35:	learn: 0.6357606	total: 1.18s	remaining: 2.1s
36:	learn: 0.6357606	total: 1.21s	remaining: 2.06s
37:	learn: 0.6357606	total: 1.25s	remaining: 2.03s
38:	learn: 0.6357606	total: 1.28s	remaining: 2s
39:	learn: 0.6357606	total: 1.31s	remaining: 1.97s
40:	learn: 0.6357606	total: 1.34s	r

...

86:	learn: 0.6449457	total: 9.67s	remaining: 12.6s
87:	learn: 0.6449197	total: 9.78s	remaining: 12.4s
88:	learn: 0.6447697	total: 9.9s	remaining: 12.3s
89:	learn: 0.6447606	total: 10s	remaining: 12.2s
90:	learn: 0.6447810	total: 10.1s	remaining: 12.1s
91:	learn: 0.6447116	total: 10.2s	remaining: 12s
92:	learn: 0.6446670	total: 10.3s	remaining: 11.9s
93:	learn: 0.6446743	total: 10.4s	remaining: 11.8s
94:	learn: 0.6447893	total: 10.6s	remaining: 11.7s
95:	learn: 0.6447832	total: 10.7s	remaining: 11.6s
96:	learn: 0.6448750	total: 10.8s	remaining: 11.5s
97:	learn: 0.6447675	total: 10.9s	remaining: 11.3s
98:	learn: 0.6448116	total: 11s	remaining: 11.2s
99:	learn: 0.6448776	total: 11.1s	remaining: 11.1s
100:	learn: 0.6447727	total: 11.2s	remaining: 11s
101:	learn: 0.6447591	total: 11.3s	remaining: 10.9s
102:	learn: 0.6450305	total: 11.4s	remaining: 10.8s
103:	learn: 0.6450513	total: 11.6s	remaining: 10.7s
104:	learn: 0.6450257	total: 11.7s	remaining: 10.6s
105:	learn: 0.6458041	total: 11.8s	

44:	learn: 0.6459984	total: 4.95s	remaining: 17.1s
45:	learn: 0.6457519	total: 5.06s	remaining: 16.9s
46:	learn: 0.6456780	total: 5.17s	remaining: 16.8s
47:	learn: 0.6458207	total: 5.29s	remaining: 16.7s
48:	learn: 0.6454364	total: 5.39s	remaining: 16.6s
49:	learn: 0.6451673	total: 5.5s	remaining: 16.5s
50:	learn: 0.6451699	total: 5.62s	remaining: 16.4s
51:	learn: 0.6449598	total: 5.73s	remaining: 16.3s
52:	learn: 0.6447352	total: 5.84s	remaining: 16.2s
53:	learn: 0.6446866	total: 5.95s	remaining: 16.1s
54:	learn: 0.6446874	total: 6.07s	remaining: 16s
55:	learn: 0.6444588	total: 6.18s	remaining: 15.9s
56:	learn: 0.6442388	total: 6.29s	remaining: 15.8s
57:	learn: 0.6442774	total: 6.4s	remaining: 15.7s
58:	learn: 0.6452272	total: 6.51s	remaining: 15.6s
59:	learn: 0.6450170	total: 6.62s	remaining: 15.4s
60:	learn: 0.6446363	total: 6.73s	remaining: 15.3s
61:	learn: 0.6444753	total: 6.84s	remaining: 15.2s
62:	learn: 0.6446311	total: 6.95s	remaining: 15.1s
63:	learn: 0.6446898	total: 7.07s	r

2:	learn: 0.6434627	total: 322ms	remaining: 21.1s
3:	learn: 0.6467273	total: 431ms	remaining: 21.1s
4:	learn: 0.6480258	total: 540ms	remaining: 21.1s
5:	learn: 0.6452758	total: 649ms	remaining: 21s
6:	learn: 0.6464273	total: 758ms	remaining: 20.9s
7:	learn: 0.6455602	total: 865ms	remaining: 20.8s
8:	learn: 0.6457714	total: 974ms	remaining: 20.7s
9:	learn: 0.6459993	total: 1.08s	remaining: 20.5s
10:	learn: 0.6462004	total: 1.19s	remaining: 20.4s
11:	learn: 0.6462153	total: 1.3s	remaining: 20.4s
12:	learn: 0.6463651	total: 1.41s	remaining: 20.3s
13:	learn: 0.6463828	total: 1.52s	remaining: 20.2s
14:	learn: 0.6467922	total: 1.63s	remaining: 20.1s
15:	learn: 0.6462826	total: 1.75s	remaining: 20.1s
16:	learn: 0.6455045	total: 1.86s	remaining: 20s
17:	learn: 0.6472436	total: 1.97s	remaining: 19.9s
18:	learn: 0.6471312	total: 2.08s	remaining: 19.8s
19:	learn: 0.6464049	total: 2.19s	remaining: 19.7s
20:	learn: 0.6469246	total: 2.3s	remaining: 19.6s
21:	learn: 0.6470269	total: 2.41s	remaining: 

164:	learn: 0.6521945	total: 18.3s	remaining: 3.89s
165:	learn: 0.6522067	total: 18.4s	remaining: 3.78s
166:	learn: 0.6520606	total: 18.6s	remaining: 3.67s
167:	learn: 0.6521275	total: 18.7s	remaining: 3.55s
168:	learn: 0.6521814	total: 18.8s	remaining: 3.44s
169:	learn: 0.6520573	total: 18.9s	remaining: 3.33s
170:	learn: 0.6521418	total: 19s	remaining: 3.22s
171:	learn: 0.6522862	total: 19.1s	remaining: 3.11s
172:	learn: 0.6526073	total: 19.2s	remaining: 3s
173:	learn: 0.6526090	total: 19.3s	remaining: 2.89s
174:	learn: 0.6525721	total: 19.4s	remaining: 2.78s
175:	learn: 0.6525849	total: 19.5s	remaining: 2.67s
176:	learn: 0.6525619	total: 19.7s	remaining: 2.56s
177:	learn: 0.6526428	total: 19.8s	remaining: 2.44s
178:	learn: 0.6526592	total: 19.9s	remaining: 2.33s
179:	learn: 0.6526524	total: 20s	remaining: 2.22s
180:	learn: 0.6526291	total: 20.1s	remaining: 2.11s
181:	learn: 0.6525610	total: 20.2s	remaining: 2s
182:	learn: 0.6525619	total: 20.3s	remaining: 1.89s
183:	learn: 0.6526064	

124:	learn: 0.7628139	total: 14.2s	remaining: 8.55s
125:	learn: 0.7636042	total: 14.4s	remaining: 8.44s
126:	learn: 0.7634977	total: 14.5s	remaining: 8.33s
127:	learn: 0.7635755	total: 14.6s	remaining: 8.21s
128:	learn: 0.7640110	total: 14.7s	remaining: 8.1s
129:	learn: 0.7647051	total: 14.8s	remaining: 7.99s
130:	learn: 0.7648350	total: 14.9s	remaining: 7.87s
131:	learn: 0.7652855	total: 15s	remaining: 7.75s
132:	learn: 0.7653897	total: 15.2s	remaining: 7.63s
133:	learn: 0.7655393	total: 15.3s	remaining: 7.52s
134:	learn: 0.7655218	total: 15.4s	remaining: 7.4s
135:	learn: 0.7661378	total: 15.5s	remaining: 7.29s
136:	learn: 0.7670052	total: 15.6s	remaining: 7.18s
137:	learn: 0.7674074	total: 15.8s	remaining: 7.08s
138:	learn: 0.7677002	total: 15.9s	remaining: 6.97s
139:	learn: 0.7682979	total: 16s	remaining: 6.85s
140:	learn: 0.7687888	total: 16.1s	remaining: 6.74s
141:	learn: 0.7690097	total: 16.2s	remaining: 6.62s
142:	learn: 0.7693748	total: 16.3s	remaining: 6.51s
143:	learn: 0.7711

In [ ]:
# Checkpoint

CAT_BOOST_MODEL = "18_ORD_UA_all_delays_catboost_fbeta_0.5.pkl"

with open(f"{SCRATCH_DIR}/{CAT_BOOST_MODEL}", "wb") as f:
    pickle.dump(model, f)

# with open(f"{SCRATCH_DIR}/{CAT_BOOST_MODEL}", "rb") as f:
#     model = pickle.load(f)

# Model Evaluation

In [9]:
# Check this model that was computed in Great Lakes
with open(f"{SCRATCH_DIR}/18_ORD_UA_all_delays_catboost_fbeta_0.5.pkl", "rb") as f:
    model = pickle.load(f)

/Users/Pat/Documents/umich/2023-09 SIADS 696/.venv/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator GridSearchCV from version 1.2.1 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
print("Best parameters: ", model.best_params_)
print("Best score: ", model.best_score_)

model.cv_results_

Best parameters:  {'depth': 10, 'iterations': 200, 'l2_leaf_reg': 1, 'learning_rate': 0.2}
Best score:  0.8449242597574239


{'mean_fit_time': array([ 20.34599447,  19.59040589,  20.95789099,  21.79668765,
         20.78378143,  20.2893187 ,  21.51725774,  22.12733049,
         20.54326248,  21.45882888,  20.86725683,  19.51882277,
         61.9249404 ,  70.92983418,  76.8025826 ,  76.27475004,
         68.27855177,  72.12645006,  74.4021616 ,  75.79627123,
         69.23503842,  76.4894269 ,  76.14262066,  77.30744524,
         25.77604465,  23.97482328,  24.58596778,  24.64137564,
         23.43752866,  25.42671342,  24.21845851,  24.50980043,
         25.52454758,  24.04928064,  25.13435454,  25.74425097,
         94.48314095, 102.72125087, 105.78670077, 107.55177698,
         93.6719883 , 102.25241084, 104.75565572, 105.58238177,
         96.52193341, 100.60151653, 107.77427073, 101.44609685,
         38.8024899 ,  36.15244498,  35.7966136 ,  37.46307893,
         36.63805189,  37.20733337,  38.06996717,  38.11028371,
         36.97242527,  36.23232799,  35.74043489,  36.77374125,
        140.40805297, 1

In [11]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    roc_curve,
    roc_auc_score,
    precision_recall_curve,
)

y_pred = model.predict(X_test)

ra_score = roc_auc_score(y_test, y_pred, average="macro", multi_class="ovr")

ra_score

NameError: name 'X_test' is not defined

In [ ]:
cm = confusion_matrix(y_test, y_pred)

print(cm)

ConfusionMatrixDisplay(
    confusion_matrix=cm, display_labels=["Not Delayed", "Delayed"]
).plot(cmap="Blues")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
y_scores = model.predict_proba(X_test)[:, 1]

precision, recall, _ = precision_recall_curve(y_test, y_scores)

plt.figure()
plt.step(recall, precision, where='post')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()


In [ ]:
fpr, tpr, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

plt.figure()
plt.plot(fpr, tpr, color="darkorange", label="ROC curve")
plt.plot([0, 1], [0, 1], color="navy", linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic")
plt.legend(loc="lower right")
plt.show()

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

accuracy, precision, recall, f1

In [ ]:
metrics_scores = {
    "Accuracy": accuracy_score(y_test, y_pred),
    "Precision": precision_score(y_test, y_pred),
    "Recall": recall_score(y_test, y_pred),
    "F1 Score": f1_score(y_test, y_pred),
}

df_metrics = pd.DataFrame(list(metrics_scores.items()), columns=["Metric", "Score"])

base = alt.Chart(df_metrics).encode(
    y=alt.Y("Metric:N", axis=alt.Axis(title="Metric")),
    x=alt.X("Score:Q", axis=alt.Axis(title="Score"), scale=alt.Scale(domain=[0, 1])),
)

bar = base.mark_bar().encode(
    color=alt.Color("Metric:N", legend=None), text=alt.Text("Score:Q", format=".2f")
)

text = base.mark_text(
    align="right",
    baseline="middle",
    color="white",
    dx=-10,
).encode(text=alt.Text("Score:Q", format=".2f"))

chart = (
    (bar + text)
    .properties(title="Evaluation Metrics", width=600, height=300)
    .configure_axis(labelFontSize=12, titleFontSize=14)
)

chart